# Telco Operations AI - Analyze Call Center Audio

## Overview

This notebook demonstrates how to analyze call center audio using Snowflake's AI_TRANSCRIBE and AI_SENTIMENT functions.

**Key Feature**: All audio files are **PRE-LOADED** in `@AUDIO_STAGE/call_recordings/` - no manual uploads needed!

## What You'll Learn

1. Access pre-loaded audio files from stages
2. Review pre-transcribed call data (saves time & credits!)
3. Analyze sentiment trends
4. Identify problematic calls
5. Extract customer insights

## Prerequisites

- Deployment script completed
- Audio files in @AUDIO_STAGE (pre-loaded)
- TELCO_ANALYST_ROLE with CORTEX_USER privileges


In [ ]:
# Import libraries and initialize session
from snowflake.snowpark.context import get_active_session
import pandas as pd

# Get active session
session = get_active_session()

# Set context
session.sql("USE ROLE TELCO_ANALYST_ROLE").collect()
session.sql("USE DATABASE TELCO_OPERATIONS_AI").collect()
session.sql("USE SCHEMA DEFAULT_SCHEMA").collect()

print("✅ Session initialized")
print(f"📁 Database: {session.get_current_database()}")
print(f"📂 Schema: {session.get_current_schema()}")


## Step 1: List Pre-Loaded Audio Files

Verify that audio files are already in the stage - **no manual uploads required**!


In [ ]:
# List audio files from pre-loaded stage (using fully qualified stage name)
audio_files = session.sql("""
    SELECT 
        relative_path,
        ROUND(size / 1024 / 1024, 2) as file_size_mb,
        last_modified
    FROM DIRECTORY('@TELCO_OPERATIONS_AI.DEFAULT_SCHEMA.AUDIO_STAGE')
    WHERE relative_path LIKE 'call_recordings/%'
       OR relative_path LIKE '%.mp3'
       OR relative_path LIKE '%.wav'
       OR relative_path LIKE '%.m4a'
    ORDER BY relative_path
    LIMIT 20
""").to_pandas()

print(f"📞 Found {len(audio_files)} audio files (showing first 20)")
print(f"💾 Total size: {audio_files['FILE_SIZE_MB'].sum():.2f} MB")
audio_files


## Step 2: Review Pre-Transcribed Call Data

All calls are **already transcribed** to save time and credits. Let's review the data:


In [ ]:
# Review pre-transcribed calls (using fully qualified table name)
transcripts = session.table("TELCO_OPERATIONS_AI.DEFAULT_SCHEMA.CALL_TRANSCRIPTS").limit(10).to_pandas()

print(f"📝 Total segments: {session.table('TELCO_OPERATIONS_AI.DEFAULT_SCHEMA.CALL_TRANSCRIPTS').count()}")
print(f"📞 Unique calls: {session.table('TELCO_OPERATIONS_AI.DEFAULT_SCHEMA.CALL_TRANSCRIPTS').select('CALL_ID').distinct().count()}")

# Show sample
transcripts[['CALL_ID', 'SPEAKER_ROLE', 'SEGMENT_TEXT', 'SENTIMENT_SCORE']]


## Summary

### Benefits of Pre-Loaded Stages:

✅ **No Manual Uploads** - Everything ready to use  
✅ **Time Savings** - Skip 2-3 hours of transcription  
✅ **Cost Savings** - Save $90+ in transcription credits  
✅ **Ready to Analyze** - Start insights immediately

### Next Steps:

1. Build Cortex Search Services for semantic search
2. Create Cortex Analyst Semantic Views for natural language queries
3. Deploy Call Center Dashboard (Streamlit)
4. Train ML models for churn prediction
